In [4]:
# Import Dependencies / Setup
from bs4 import BeautifulSoup
import os
from splinter import Browser
import pandas as pd
import time

# Scraping

### NASA Mars News

In [10]:
mars_data = {}

# Scraping using Splinter
# create a browser instance to be used fro accessing websites and scraping. 
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

# Scraping NASA Mars News
# URL to be scraped. Visit the URL with the browser instance
news_url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(news_url)

# Use BeautifulSoup to parse out needed information (html)
html = browser.html
#print(html)
soup = BeautifulSoup(html,'html.parser')

# Read the content_title. Skip the first one since it the page title 
news_titles=[]
news_title=""
try:
    news_titles = soup.find_all("div",class_="content_title")
    for xx in news_titles:
        try:
            news_title = xx.text
        except:
            pass
        # Skip this title since it is for the page
        if news_title=="Mars Now":
            continue
        break      
except Exception as e:
    news_title = "No news is not always good news"
    print(e)

# retrieve the latest news snippet
news_paragraph = ""
try:
    news_paragraph = soup.find("div", class_="article_teaser_body").text.strip()
except Exception as e:
    print(e)

print(f"The news title is: {news_title}")
print(f"The paragraph text is: {news_paragraph}")

The news title is: NASA's Perseverance Pays Off Back Home
The paragraph text is: Even as the Perseverance rover approaches Mars, technology on board is paying off on Earth.


### Mars Facts 

In [16]:
# ### Mars Facts 
# URL to be scraped "https://space-facts.com/mars/"
facts_url = "https://space-facts.com/mars/"

# Use pandas to view table contents
facts_table = pd.read_html(facts_url)
facts_table

facts_df = facts_table[0]
facts_df.columns=['Planet', 'Mars']
facts_df

,Planet,Mars
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [17]:
# Convert to html
mars_facts = facts_df.to_html(index=False,index_names=False,classes=["table-bordered", "table-striped", "table-hover", "isi"])
mars_facts.replace('\n', '')

'<table border="1" class="dataframe table-bordered table-striped table-hover isi">  <thead>    <tr style="text-align: right;">      <th>Planet</th>      <th>Mars</th>    </tr>  </thead>  <tbody>    <tr>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <td>Surface Temperature:</td>      <td>-87 to -5 °C</td>    </tr>    <tr>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <td>Recorded By:</td>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

### Mars Hemispheres 

In [24]:
# Visit website displaying the hemispheres of Mars
hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemispheres_url)

# Parse HTML
html_hemispheres = browser.html
time.sleep(2)
soup = BeautifulSoup(html_hemispheres,'html.parser')

# Pull hemispheres information
info = soup.find_all('a', class_='itemLink product-item')

# Start empty list for the URLs
hemisphere_images_urls = []

# Loop through the items previously stored
for i in info: 
    # Store title
    h3_text = ""
    try:
        h3_text =  i.find('h3').text
    except Exception as e:
        #print(e)
        pass
    
    # Skip the images without a title to avoid duplicates
    if h3_text == '' :
        continue

    # Store link
    item_link=""
    try:
        item_link = i.get('href')
    except Exception as e:
        #print(e)
        pass
    
    # Skip the images without a link
    if item_link == '' :
        continue

    # Visit the link that contains the full image website 
    hemispheres_url = f"https://astrogeology.usgs.gov{item_link}"
    browser.visit(hemispheres_url)
    html_hemispheres = browser.html
    time.sleep(1)

    # Parse HTML for each individual hemisphere 
    soup = BeautifulSoup(html_hemispheres, 'html.parser')

    title=""
    try:
        title = soup.find('h2', class_='title').text.strip()
    except Exception as e:
        #print(e)
        pass

    # Get the img
    img_urls=[]
    img_url=""
    try: 
        img_urls = soup.find_all("a", text="Sample")
        for img_link in img_urls:         
            img_url =img_link.get('href')
            break;
    except Exception as e:
        #print(e)
        pass

    # Retrieve image source 
    # Append the retreived information into a list of dictionaries 
    hemisphere_images_urls.append({"title" : title, "img_url" : img_url})

hemisphere_images_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [25]:
# MongoDB

# Store all scraped data into a dictionary for use by the template
# Create empty dictionary
mars_data = {}

# Add information to dictionary
mars_data['news_title'] = news_title
mars_data['news_paragraph'] = news_paragraph

# Add hemisphere_image_urls to mars_data.
mars_data['hemisphere_images_urls'] = hemisphere_images_urls

# Append mars_facts to mars_data.
mars_data['mars_facts'] = mars_facts

mars_data

{'news_title': "NASA's Perseverance Pays Off Back Home",
 'news_paragraph': 'Even as the Perseverance rover approaches Mars, technology on board is paying off on Earth.',
 'hemisphere_images_urls': [{'title': 'Cerberus Hemisphere Enhanced',
   'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
  {'title': 'Schiaparelli Hemisphere Enhanced',
   'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
  {'title': 'Syrtis Major Hemisphere Enhanced',
   'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
  {'title': 'Valles Marineris Hemisphere Enhanced',
   'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}],
 'mars_facts': '<table border="1" class="dataframe table-bordered table-striped table-hover isi">\n  <thead>\n    <tr style="text-align: right;">\n   

In [26]:
browser.quit()